In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def close_pop_up(driver):
    try:
        # Assume pop-up has a close button with a certain class or id
        pop_up_close_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@class='close']"))
        )
        pop_up_close_button.click()
        time.sleep(1)  # Give it some time to close
    except Exception as e:
        # If no pop-up, or pop-up couldn't be closed, just pass
        pass


# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        close_pop_up(driver)
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages(driver):
    for page in range(1, 6):  # There are 5 pages
        try:
            load_page(driver, URL)
            
            if page >= 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Initialize the driver outside the loop
driver = initialize_driver()

# Scrape routes and details from all pages
scrape_all_pages(driver)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('uttarpradesh_bus_details.csv', index=False)

# Close the driver after scraping is done
if driver:
    driver.quit()


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/lucknow-to-aligarh: Message: 
Stacktrace:
0   chromedriver                        0x0000000100e20500 cxxbridge1$str$ptr + 1917112
1   chromedriver                        0x0000000100e18890 cxxbridge1$str$ptr + 1885256
2   chromedriver                        0x0000000100a28538 cxxbridge1$string$len + 89424
3   chromedriver                        0x0000000100a6c878 cxxbridge1$string$len + 368784
4   chromedriver                        0x0000000100aa6b7c cxxbridge1$string$len + 607124
5   chromedriver                        0x0000000100a61374 cxxbridge1$string$len + 322444
6   chromedriver                        0x0000000100a61fc4 cxxbridge1$string$len + 325596
7   chromedriver                        0x0000000100de7d2c cxxbridge1$str$ptr + 1685732
8   chromedriver                        0x0000000100dec530 cxxbridge1$str$ptr + 1704168
9   chromedriver                        0x0000000100dcce08 cxxbridge1$str$ptr

In [7]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BLY0155,Ordinary Non AC Seater 2+3,12:00,05h 30m,17:30,3.3,INR 448,52 Seats available
1,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - HPR0006,Ordinary Non AC Seater 2+3,12:00,06h 30m,18:30,3.3,INR 444,52 Seats available
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - GRH0127,Ordinary Non AC Seater 2+3,12:00,06h 30m,18:30,3.3,INR 449,52 Seats available
3,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,,,,,,,,
4,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
691,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - MNP0132,Ordinary Non AC Seater 2+3,23:32,04h 40m,04:12,3.3,INR 371,52 Seats available
692,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,Non A/C Seater / Sleeper (2+1),22:00,07h 30m,05:30,0,INR 500,9 Seats available
693,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,A/C Seater/Sleeper (2+1),22:15,06h 30m,04:45,0,INR 600,9 Seats available
694,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,Chaturvedi Bus Service,Non A/C Seater / Sleeper (2+1),22:30,06h 50m,05:20,0,INR 500,8 Seats available
